In [ ]:
!pip install torch torchvision transformers scikit-learn nltk

import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from nltk.stem import PorterStemmer
import random
import nltk
from nltk import pos_tag, word_tokenize

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

In [ ]:
random.seed(42)
torch.manual_seed(42)

stemmer = PorterStemmer()
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_text(text):
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

def grade_to_class(grade):
    return int(grade * 2)

def class_to_grade(class_idx):
    return class_idx / 2.0

def contains_grammar_mistakes(text):
    tokens = word_tokenize(text)
    tags = [tag for word, tag in pos_tag(tokens)]

    patterns = [
        ['JJ', 'JJ'],
        ['NN', 'VB'],
    ]

    for pattern in patterns:
        for i in range(len(tags) - len(pattern) + 1):
            if tags[i:i+len(pattern)] == pattern:
                return True
    return False

In [ ]:
sample_answers = [
    "Supervised learning requires labeled data, which means each example in the dataset is paired with the correct output. It uses this labeled data to train the model and make predictions on new, unseen data. Unsupervised learning, on the other hand, deals with unlabeled data. It tries to find patterns or relationships in the data without any prior knowledge of the correct output.",
    "Supervised learning uses labeled data for training, whereas unsupervised learning uses unlabeled data.",
    "In supervised learning, we have an algorithm that learns from labeled data, while unsupervised learning is where the algorithm learns from unlabeled data.",
    "Supervised is when the data is labeled. Unsupervised is when it's not.",
    "Supervised learning is where you train the model by presenting it with input and the correct output, and it learns by comparing its actual output with the correct outputs to find errors. Unsupervised learning is where a model is trained using no historical data."
]

grades = [5.0, 3.5, 4.0, 2.5, 4.5]

answers, grades_dataset = zip(*random.choices(list(zip(sample_answers, grades)), k=1000))
grades_dataset = [grade + random.uniform(-0.5, 0.5) for grade in grades_dataset]
preprocessed_answers = [preprocess_text(ans) for ans in answers]
final_answers = preprocessed_answers * 2
final_grades = grades_dataset * 2

train_answers, test_answers, train_grades, test_grades = train_test_split(
    final_answers, final_grades, test_size=0.2, random_state=42
)

train_grades = [grade_to_class(grade) for grade in train_grades]
test_grades = [grade_to_class(grade) for grade in test_grades]

train_inputs = tokenizer(train_answers, padding=True, truncation=True, return_tensors="pt", max_length=128)
test_inputs = tokenizer(test_answers, padding=True, truncation=True, return_tensors="pt", max_length=128)

train_labels = torch.tensor(train_grades).long()
test_labels = torch.tensor(test_grades).long()

train_dataset = TensorDataset(train_inputs.input_ids, train_inputs.attention_mask, train_labels)
test_dataset = TensorDataset(test_inputs.input_ids, test_inputs.attention_mask, test_labels)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4)

In [ ]:
def ensemble_predict(models, tokenized_input):
    models = [model.eval() for model in models]
    total_prediction = torch.zeros(tokenized_input["input_ids"].shape[0], 11)  # For 11 classes
    with torch.no_grad():
        for model in models:
            outputs = model(**tokenized_input)
            total_prediction += outputs.logits
    return torch.argmax(total_prediction, dim=1)

model1 = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=11)
model2 = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=11)

loss_fn = nn.CrossEntropyLoss()
optimizer1 = AdamW(model1.parameters(), lr=2e-5)
optimizer2 = AdamW(model2.parameters(), lr=2e-5)

In [ ]:
for model, optimizer in [(model1, optimizer1), (model2, optimizer2)]:
    for epoch in range(3):
        model.train()
        for batch in train_loader:
            input_ids, attention_mask, labels = batch
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def get_feedback(student_answer, keywords=None, point_based=False):
    # Check for out-of-context answers
    if is_out_of_context(student_answer, sample_answers, keywords):
        feedback = "Your answer is out of context or not relevant to the question."
        return feedback, 0.0

    preprocessed_student_answer = preprocess_text(student_answer)
    tokenized_student_answer = tokenizer(preprocessed_student_answer, padding=True, truncation=True, return_tensors="pt", max_length=128)

    predicted_class = ensemble_predict([model1, model2], tokenized_student_answer).item()
    predicted_score = class_to_grade(predicted_class) / 2.5  # Adjust to 0-2 scale

    # Deduct marks for very short answers
    if len(student_answer.split()) < 5:
        predicted_score = max(predicted_score - 0.5, 0)

    # Deduct marks for grammar mistakes
    if contains_grammar_mistakes(student_answer):
        predicted_score = max(predicted_score - 0.25, 0)

    # Point-based scoring
    if point_based:
        points = [is_out_of_context(point, sample_answers) for point in student_answer.split('.')]
        valid_points = sum([1 for point in points if not point])
        predicted_score = min(valid_points, 2.0)

    feedback_thresholds = [
        (2.0, "Perfect! Your answer is spot on."),
        (1.5, "Very good! You've covered most of the key points."),
        (1.0, "Good job! You've covered some of the main points."),
        (0.5, "You've touched upon the topic, but you need to elaborate more."),
        (0.0, "Your answer doesn't address the question.")
    ]

    feedback = "Please review your answer."
    for threshold, message in feedback_thresholds:
        if predicted_score >= threshold:
            feedback = message
            break

    return feedback, predicted_score

# Keywords and point-based instructions for the question
keywords_for_question = ["supervised", "unsupervised", "learning"]
point_based_question = True

student_answer = input("Enter your response: ")
feedback, score = get_feedback(student_answer, keywords_for_question, point_based_question)
print(f"Predicted Grade: {score}")
print("Feedback:", feedback)


Enter your response: Unsupervised learning:- No help at all is provided, the model should learn everything on its own, Supervised learning:- Humans oversee and help the model to learn
Predicted Grade: 1
Feedback: Good job! You've covered some of the main points.
